In [1]:
import logging
from lisa.conf import LisaLogging
LisaLogging.setup()

2018-09-17 17:44:33,719 INFO    : root         : Using LISA logging configuration:
2018-09-17 17:44:33,719 INFO    : root         :   /home/dourai01/Data/Git/lisa/py3k/logging.conf


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from lisa.env import TestEnv

# Import support for Android devices
from lisa.android import Screen, Workload

# Support for trace events analysis
from lisa.trace import Trace

# Suport for FTrace events parsing and visualization
import trappy
import pandas as pd

from lisa.wlgen.rta import RTA, Periodic, Ramp
from time import sleep

from IPython.display import display

Populating the interactive namespace from numpy and matplotlib


In [4]:
linux_hikey = {
    "platform" : "linux",
    "board" : "hikey",
    "host": "pwrsft-hikey620-1",
    #"username" : "root",
    #"password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"],
    "rtapp-calib" :  {
        "0": 302, "1": 302, "2": 302, "3": 302, "4": 136, "5": 136, "6": 136, "7": 136
     }
}

In [ ]:
linux_juno = {
    "platform" : "linux",
    "board" : "juno",
    "host": "192.168.0.1",
    "username" : "root",
    "password" : "root",
    "modules" : ["sched", "cgroups", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [ ]:
android_hikey = {
    "platform" : "android",
    "board" : "hikey960",
    "modules" : ["sched", "hotplug"],
    "tools" : ["taskset", "rt-app"]
}

In [5]:
te = TestEnv(linux_hikey)
target = te.target

2018-09-17 17:44:39,671 INFO    : TestEnv      : Using base path: /home/dourai01/Data/Git/lisa/py3k
2018-09-17 17:44:39,672 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-09-17 17:44:39,675 INFO    : TestEnv      : Devlib modules to load: ['cpufreq', 'sched', 'cpuidle', 'cgroups', 'hotplug']
2018-09-17 17:44:39,676 INFO    : TestEnv      : Connecting linux target:
2018-09-17 17:44:39,676 INFO    : TestEnv      :   username : root
2018-09-17 17:44:39,677 INFO    : TestEnv      :   password : 
2018-09-17 17:44:39,677 INFO    : TestEnv      :       host : pwrsft-hikey620-1
2018-09-17 17:44:39,678 INFO    : TestEnv      : Connection settings:
2018-09-17 17:44:39,678 INFO    : TestEnv      :    {'username': 'root', 'password': '', 'host': 'pwrsft-hikey620-1'}
2018-09-17 17:44:44,809 INFO    : TestEnv      : Initializing target workdir:
2018-09-17 17:44:44,811 INFO    : TestEnv      :    /root/devlib-target
2018-09-17 17:44:45,603 INFO    : CGroups      : Availabl

### Generics

In [6]:
from lisa.tests.kernel.scheduler.eas_behaviour import *

In [7]:
tests = [OneSmallTask, ThreeSmallTasks, TwoBigTasks, TwoBigThreeSmall, RampUp, RampDown, EnergyModelWakeMigration]

In [ ]:
res = {}

for test in tests:
    bundle = test.from_target(te)
    res[test.__name__] = bundle

In [ ]:
for test_name, bundle in res.items():
    print(test_name)
    print("---")
    print("test_slack: {}".format("PASSED" if bundle.test_slack() else "FAILED"))
    print("test_task_placement: {}".format("PASSED" if bundle.test_task_placement() else "FAILED"))
    print(bundle.test_task_placement())
    print()

### CPU sanity check

In [ ]:
from lisa.tests.kernel.misc.capacity_sanity import CapacitySanityCheck

In [ ]:
bundle = CapacitySanityCheck.from_target(te)

In [ ]:
if bundle.test_capacity_sanity():
    print("PASSED")
else:
    print("FAILED")

print(bundle.test_capacity_sanity())

### Hotplug

In [ ]:
from lisa.tests.kernel.hotplug.torture import HotplugTorture

In [ ]:
bundle = HotplugTorture.from_target(te)

In [ ]:
bundle.test_cpus_alive().passed

In [ ]:
bundle.test_cpus_alive().metrics

In [ ]:
bundle.test_target_alive().passed